In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1)
import sys
import sklearn
import sklearn.ensemble
%load_ext autoreload
%autoreload 2
from anchor import utils
from anchor import anchor_tabular
from int_met import *

C:\Users\youss\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
dataset_folder = 'Datasets/'
dataset = utils.load_dataset('mortality', balance=True, dataset_folder=dataset_folder)

In [3]:
dataset.ori_test[0]

array(['40.27378464', 'White', 'Female', '7', '132', '0.927777767',
       'Chest Pain', 'No', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'No'],
      dtype=object)

In [4]:
explainer = anchor_tabular.AnchorTabularExplainer(dataset.class_names, dataset.feature_names, dataset.data, dataset.categorical_names)
explainer.fit(dataset.train, dataset.labels_train, dataset.validation, dataset.labels_validation)

In [5]:
c = sklearn.ensemble.RandomForestClassifier(n_estimators=100, n_jobs=5, random_state= 46)
c.fit(explainer.encoder.transform(dataset.train), dataset.labels_train)
predict_fn = lambda x: c.predict(explainer.encoder.transform(x))
print('Train', sklearn.metrics.accuracy_score(dataset.labels_train, predict_fn(dataset.train)))
print('Test', sklearn.metrics.accuracy_score(dataset.labels_test, predict_fn(dataset.test)))
print('Train', sklearn.metrics.roc_auc_score(dataset.labels_train, predict_fn(dataset.train)))
print('Test', sklearn.metrics.roc_auc_score(dataset.labels_test, predict_fn(dataset.test)))

Train 1.0
Test 1.0
Train 1.0
Test 1.0


In [15]:
import pickle
pickling_on = open("rfmortanc.pickle","wb")
pickle.dump(c, pickling_on)
pickling_on.close()

In [6]:
exp = explainer.explain_instance(dataset.test[0], c.predict, threshold=0.95)

In [34]:
exp_fn_bulk = lambda x: np.array([np.array(explainer.explain_instance(x[idx], c.predict, threshold=0.95).features()) for idx in range(len(x))])

In [7]:
dataset.feature_names

['age',
 'racewb',
 'sex',
 'mets_achieved',
 'resting_systolic',
 'percent_hr_achieved',
 'reason',
 'diuretic',
 'htnmed',
 'htn',
 'dm',
 'famhx',
 'smoke',
 'afib2']

In [8]:
exp.names()

['smoke = Yes']

In [9]:
exp.features()

[12]

In [35]:
exp1 = exp_fn_bulk(dataset.test[:3])

In [36]:
exp1

array([array([0, 9, 8, 2, 4]), array([ 3, 10,  8,  4,  2, 12]),
       array([ 0,  9, 10,  5, 11])], dtype=object)

In [40]:
num_feats = len(dataset.feature_names)
imp = np.zeros(num_feats).astype(int)

In [47]:
for i in range(len(dataset.test)):
    exp = explainer.explain_instance(dataset.test[i], c.predict, threshold=0.95).features()
    for j in exp:
        imp[j] = imp[j] + 1

In [48]:
imp

array([685, 140, 253, 698, 267, 275, 283, 117, 236, 209, 197, 225, 309,
       132])

In [62]:
order = np.argsort(-imp)
a = list(np.array(feats)[order])

In [68]:
a

['mets_achieved',
 'age',
 'smoke',
 'reason',
 'percent_hr_achieved',
 'resting_systolic',
 'sex',
 'htnmed',
 'famhx',
 'htn',
 'dm',
 'racewb',
 'afib2',
 'diuretic']

In [52]:
dataset.ori_test.shape

(836, 15)

In [96]:
import copy
import pandas as pd
feats = copy.deepcopy(dataset.feature_names)
feats.append('death')
ori_data = pd.DataFrame(dataset.ori_test, columns=feats)

In [76]:
a

['mets_achieved',
 'age',
 'smoke',
 'reason',
 'percent_hr_achieved',
 'resting_systolic',
 'sex',
 'htnmed',
 'famhx',
 'htn',
 'dm',
 'racewb',
 'afib2',
 'diuretic']

In [99]:
z

['death',
 'mets_achieved',
 'age',
 'smoke',
 'reason',
 'percent_hr_achieved',
 'resting_systolic',
 'sex',
 'htnmed',
 'famhx',
 'htn',
 'dm',
 'racewb',
 'afib2',
 'diuretic']

In [100]:
view1 = ori_data.reindex(columns=z)

In [104]:
preds =  predict_fn(dataset.test)

In [97]:
feats

['age',
 'racewb',
 'sex',
 'mets_achieved',
 'resting_systolic',
 'percent_hr_achieved',
 'reason',
 'diuretic',
 'htnmed',
 'htn',
 'dm',
 'famhx',
 'smoke',
 'afib2',
 'death']

In [105]:
view1.insert(loc=1,column='prediction',value=preds)

In [ ]:
ori_data

In [107]:
yesmap={0:'No', 1:'Yes'}
yes_no = lambda x: yesmap[x] 

In [108]:
view1['prediction']=view1['prediction'].apply(yes_no)

In [102]:
ori_data

,age,racewb,sex,mets_achieved,resting_systolic,percent_hr_achieved,reason,diuretic,htnmed,htn,dm,famhx,smoke,afib2,death
0,40.27378464,White,Female,7,132,0.927777767,Chest Pain,No,No,No,No,Yes,Yes,No,No
1,52.56399536,Black,Male,7,156,0.910179615,Chest Pain,No,Yes,Yes,Yes,No,Yes,No,No
2,49.34976196,White,Male,-1,130,0.888888896,Chest Pain,No,No,No,No,Yes,Yes,No,No
3,65.53045654,Black,Female,7,120,0.870129883,Chest Pain,Yes,Yes,Yes,No,No,No,No,No
4,77.66461182,Black,Male,4.599999905,102,0.936619699,Chest Pain,Yes,Yes,Yes,No,No,Yes,No,Yes
5,71.25530243,White,Female,7,142,0.872483194,Shortness of Breath,No,Yes,Yes,Yes,Yes,Yes,Yes,No
6,59.19233322,Black,Female,7,162,0.919254661,Palpitation,No,No,Yes,No,Yes,No,No,No
7,45.94113541,Black,Male,13,108,0.988505721,Chest Pain,No,Yes,Yes,Yes,No,No,No,Yes
8,41.67830276,Black,Female,7,144,0.95505619,Chest Pain,No,Yes,Yes,No,No,No,No,No
9,67.4907608,Other,Male,10,132,0.947712421,Rule out Ischemia,No,No,No,Yes,No,No,No,Yes


In [111]:
rm_comma = lambda x: x.replace(',','')

In [112]:
view1['reason']=view1['reason'].apply(rm_comma)

In [114]:
view1.to_csv('amortanchor.csv',index=False)

In [ ]:
view1